# Named Entity Recognition
Theoretical explanation is present in this blogpost [link](). The project aims to use Bidirectional-LSTM with Conditional Random Fields along with static word embeddings to perform Named Entity Recognition task based off of the CoNLL-2003 dataset. This will be done step by step from downloading and preparing the dataset, embeddings, to training and validating the performance.

## 1. Dataset downloading and preprocessing
The first step is to download both the dataset which contains the tagged information as well as the static embeddings GloVe, which has vectors with embedded meanings for words. Since BiLSTM cannot generated contextualised vectors like BERT and other transformer models do (a natural improvement), we fallback to using GloVe.

### 1.1 Downloading GloVe Embeddings
We will download the 100 dimensional variant of GloVe (there's a 300 dimensional variant which is heavy but more accurate). This is available for your exploration at [Stanford NLP Github](https://github.com/stanfordnlp/GloVe/releases). The following script uses utility function to download this automatically and unzips to `src/data` folder.

In [1]:
from util.download import download_glove_embeddings

In [2]:
download_glove_embeddings()

GloVe embeddings already exist. Skipping download.
